In [1]:
from diffusers import StableDiffusionPipeline, StableDiffusionXLPipeline

Info: `UmerDebugLogger` created. This is a logging class that will be deleted when the PR to integrate ControlNet-XS is done.


In [2]:
sdxl_pipe = StableDiffusionXLPipeline.from_single_file('weights/sd/sdxl/sd_xl_base_1.0_0.9vae.safetensors')
sd_pipe = StableDiffusionPipeline.from_single_file('weights/sd/sd21/v2-1_512-ema-pruned.ckpt')

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .
/home/diffusers/src/diffusers/configuration_utils.py:135: FutureWarning: Accessing config attribute `requires_safety_checker` directly via 'StableDiffusionPipeline' object attribute is deprecated. Please access 'requires_safety_checker' over 'StableDiffusionPipeline's config object instead, e.g. 'scheduler.config.requires_safety_checker'.
  depre

In [3]:
sdxl = sdxl_pipe.unet
sd21 = sd_pipe.unet

In [10]:
def print_if_uses_linear_projection(down_blocks):
    for di, d in enumerate(down_blocks):
        if not hasattr(d, 'attentions'): continue      
        for ai, a in enumerate(d.attentions):
            print(f'Block {di}/Attn {ai}: use_linear_projection = {a.use_linear_projection}')

In [11]:
print_if_uses_linear_projection(sd21.down_blocks)

Block 0/Attn 0: use_linear_projection = True
Block 0/Attn 1: use_linear_projection = True
Block 1/Attn 0: use_linear_projection = True
Block 1/Attn 1: use_linear_projection = True
Block 2/Attn 0: use_linear_projection = True
Block 2/Attn 1: use_linear_projection = True


In [12]:
print_if_uses_linear_projection(sdxl.down_blocks)

Block 1/Attn 0: use_linear_projection = True
Block 1/Attn 1: use_linear_projection = True
Block 2/Attn 0: use_linear_projection = True
Block 2/Attn 1: use_linear_projection = True
